# Testing Your Amazon Bedrock AgentCore Agent

This notebook demonstrates how to test your deployed calculator agent using the AWS SDK.
Make sure you have deployed your agent using `agentcore launch` before running these tests!

In [ ]:
# Import required libraries for AWS interaction
import boto3
import json
from botocore.exceptions import ClientError

# Create a boto3 session - you can specify your AWS profile here if needed
#boto3_client = boto3.session.Session(profile_name='agentcoreuser')
# For default profile, you can use: 
boto3_client = boto3.session.Session()
region = boto3_client.region_name

def invoke_agent(prompt, session_id, agent_arn, region):
    """
    Invoke your deployed AgentCore calculator agent with a math question.
    
    Args:
        user_id (str): Unique identifier for the user
        prompt (str): The math question or prompt to send to the agenty
        session_id (str): Session ID to maintain conversation context (must be 33+ chars)
        agent_arn (str): The ARN of your deployed agent (get this from 'agentcore status')
        region (str): AWS region where your agent is deployed
    
    Returns:
        dict: The agent's response containing the calculation result
    """
    try:
        # Initialize the Bedrock AgentCore client
        agent_core_client = boto3_client.client('bedrock-agentcore', region_name=region)
        
        # Prepare the payload with your math question
        payload = json.dumps({"prompt": prompt}).encode()

        # Call the agent with your math question
        response = agent_core_client.invoke_agent_runtime(
            agentRuntimeArn=agent_arn,
            runtimeSessionId=session_id,
            payload=payload,
            qualifier="DEFAULT"
        )
        
        # Process and display the response
        response_body = response['response'].read()
        response_data = json.loads(response_body)
        print("🧮 Agent Response:", response_data)
        return response_data
        
    except ClientError as e:
        print(f"❌ Error calling agent: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

## Test Your Calculator Agent

Before running the cell below, make sure to:
1. Replace `YOUR_AGENT_ARN_HERE` with your actual agent ARN from `agentcore status`
2. Update the region if your agent is deployed in a different region
3. Try different math problems to test the calculator functionality!

In [ ]:
# Configuration - Update these values with your agent details
prompt = "What is 25 plus 37?"  # Try different math problems!
session_id = "calculator-session-2024-test-12345-67890-abcdef"  # Must be 33+ characters

# 🚨 IMPORTANT: Replace this with your actual agent ARN from 'agentcore status'
agent_arn = "YOUR-ARN"

# Test the calculator agent
print("🚀 Testing calculator agent...")
print(f"📝 Question: {prompt}")
print("\n" + "="*50)

result = invoke_agent(prompt, session_id, agent_arn,region)